## Target- urgent withdrawn by nurse(within 24hr)
* shift_applications status == 'cancelled' + shift_application prevStatus == 'confirmed'


### IDs -> only look at shifts application, leave each shifts after
* shift_applications id
* shift_applications user_id
* shift_applications shift_id


### Numeric Field
* shifts rate
* shifts net_pay: rate - nursedash profit
* shift_applications distance
* shift break time
* user withdrawn times: 
    Problem: 
        1. We don't know in reality that you will withdraw your previous one.
        2. cannot be used to predict new user
* user applied times:
* prev SA/CW rate


## Time
### shift, shift app time
* S_Create (shift created) => SA_Create (application created) => CWTime (comfim withdrawn) => Start_Time (shift start)
* S_Create: shift create time
* SA_Create: shift application create time
* CWTime: shift withdrawn time, whole confirmed withdrawn population (include within 24hr and > 24hr)
* Start_Time: shift start time
##### calculated field, graph in hours
* S_Create2SA_Create: shift application create time - shift create time
* S_Create2Start_Time: shift start time - shift create time
        1. has negative values. delete maybe?
* SA_Create2Start_Time: shift start time - shift application create time
        1. has negative values. delete maybe?

### User
* user approvedAt:
* user createdAt:
#### Calculated field
* U_Create2U_approved: User approve time - User create time
* U_Create2now: now - User create time, in months(seconds/2629746)
* U_Approve2now: now - User approve time


### Categorical
* shifts role: position name + type
* facilities name = facilities short_name
* facilities areaId = facilities areaName
* withdrawnInfo_key, withdrawnInfo_value
* facilities segmentName: Senior Living = 1, Healthcare = 0
* Users enableNotifications:
* Users emailNotifications:
* Users appNotifications:
* Users allowedNotifications:


### ?
* facilities allowedQualification
* facilities createdAt
* facilities rates
* nurse shift withdrawn by admin, when policy start, how many withdrawn by admin
* shifts = unit
* role
* users updateAt ?
* users relationToFacility

# Concat data

In [344]:
# import pandas as pd
# import numpy as np

# df1 = pd.read_csv('0-2 new.csv')
# df2 = pd.read_csv('2-5.csv')
# df3 = pd.read_csv('5-9.csv')
# df4 = pd.read_csv('9-12.csv')
# df5 = pd.read_csv('12-18.csv')

# main = pd.concat([df1, df2, df3, df4, df5], axis = 0)

# main.to_csv('main.csv')

# Data Cleaning

In [345]:
df = pd.read_csv('main.csv').drop(columns = ['Unnamed: 0'])
df.columns

Index(['id', 'user_id', 'shift_id', 'status', 'createdAt',
       'hasNurseCheckEvent', 'hasBreakTime', 'prevStatus', 'distance', 'id.1',
       'facility_id', 'description', 'start_time', 'assigned_nurse_id', 'rate',
       'net_pay', 'unit', 'type', 'createdAt.1', 'qualifications', 'breakTime',
       'prevStatus.1', 'id.2', 'name', 'short_name', 'createdAt.2', 'rates',
       'S_Create', 'SA_Create', 'S_create2SA_Create', 'Start_Time',
       'SA_Create2Start_Time', 'S_Create2Start_Time', 'CW_Time',
       'CW_Time2Start_Time', 'SA_Create2CW_Time', 'S_Create2CW_Time',
       'allowedQualifications', 'areaId', 'areaName', 'segmentName',
       'U_create2now', 'U_approve2now', 'U_create2U_approve',
       'withdrawnInfo_key', 'withdrawnInfo_value'],
      dtype='object')

In [325]:
# fill these column with 0, no cw make these column nan
df[['CW_Time', 'SA_Create2CW_Time', 'S_Create2CW_Time']] = df[['CW_Time', 'SA_Create2CW_Time', 'S_Create2CW_Time']].fillna(0)

# There are some negative numbers in these column. Target all = 0, drop for now
df = df[(df["S_Create2Start_Time"]>0) & (df["SA_Create2Start_Time"]>0)]

# sort by time that shift was created
df = df.sort_values(by = 'createdAt').reset_index(drop = True)

In [326]:
# what are prevstatus column like when status = confirmed
df[df['status'] == 'confirmed']['prevStatus'].value_counts()

selected       35786
unavailable       10
applied            1
cancelled          1
Name: prevStatus, dtype: int64

In [327]:
# what are status column like when prevstatus = confirmed
df[df['prevStatus'] == 'confirmed']['status'].value_counts()

withdrawn    25732
cancelled     6343
applied          1
Name: status, dtype: int64

# Feature Engineering

### Target: urgent withdrawn as 1

In [328]:
def CW_by_nurse(row):
    if row['status']=='withdrawn' and row['prevStatus'] == 'confirmed':
        if row['withdrawnInfo_value'] == 'nurse':
            if row['CW_Time2Start_Time'] < 0 and row['CW_Time2Start_Time'] >= -24:
                return 1
            else:
                return 0
        else:
            return 0
    else:
        return 0
    
df['target'] = df.apply (lambda row: CW_by_nurse(row), axis=1)

In [329]:
df['target'].value_counts()

0    165745
1      4372
Name: target, dtype: int64

###  Count previsous shift application

In [330]:
# track by using dictionary
count_prev_SA = []
test = df[['user_id', 'createdAt']]

# create SA dictionary, set all value = 0 -> dramatically reduce computational cost
uid_library = list(pd.unique(test['user_id']))
sa_dict = {} 
for uid in uid_library:
    sa_dict.update({uid: 0}) 
    
for i, v in enumerate(test['user_id']):
    sa_dict[v] += 1
    count_prev_SA.append(sa_dict[v]-1)
 
df['count_prev_SA'] = count_prev_SA

###  Count previsous urgent withdrawns

In [331]:
# track by using dictionary
count_prev_CW = []
test = df[['user_id', 'count_prev_SA', 'createdAt', 'target']]

# create cw dictionary, set all value = 0 -> dramatically reduce computational cost
uid_library = list(pd.unique(test['user_id']))
cw_dict = {} 
for uid in uid_library:
    cw_dict.update({uid: 0}) 
    
# fill dictionary and fill cw count
for i, v in enumerate(test['user_id']):
    if test['target'][i] == 1:
        cw_dict[v] += 1
        count_prev_CW.append(cw_dict[v]-1)
    else:
        count_prev_CW.append(cw_dict[v])
        
df['count_prev_CW'] = count_prev_CW

### Count previsous urgent withdrawns/Count previsous shift applications

In [332]:
df['prev_CW/SA_rate'] = df['count_prev_CW']/df['count_prev_SA']

# fill nan with 0, happend bc 0/0, meaning rate = 0
df['prev_CW/SA_rate'].fillna(0,inplace=True)

### previous withdrawn times previous apply

In [333]:
df['prev_CW x SA_rate'] = df['count_prev_CW']*df['count_prev_SA']

### 2 Type dummy: RN, LVN + LPN , Rest


In [334]:
df['type'].value_counts()

CNA             87175
STNA            40401
LVN             16397
RN              12020
LPN              6871
CMA/Med-Tech     6282
Tech              971
Name: type, dtype: int64

In [335]:
df['type_RN'] = df.apply(lambda row:1 if row['type'] == 'RN' else 0, axis = 1)
df['type_LVN+LPN'] = df.apply(lambda row: 1 if row['type'] == 'LVN' or row['type'] == 'LPN' else 0, axis = 1)

### 1 SegementName dummy: Senior Living, Healthcare

In [336]:
df['segmentName'].value_counts()

Senior Living    148632
Healthcare        21485
Name: segmentName, dtype: int64

In [337]:
def create_segmentName_dummy(row):
    if row['segmentName']=='Senior Living':
        return 1
    else:
        return 0
    
df['segmentName_d'] = df.apply(lambda row: create_segmentName_dummy(row), axis=1)

### 5 Facility Area Name Dummy

In [338]:
df['areaName'].value_counts()

Houston           85426
Northeast Ohio    47491
DFW               27376
Austin             6080
San Antonio        3599
Cincinnati          145
Name: areaName, dtype: int64

In [339]:
# Houston
df['areaName_houston'] = df.apply(lambda row: 1 if row['areaName'] == 'Houston' else 0, axis = 1)

# Northeast Ohio
df['areaName_no'] = df.apply(lambda row: 1 if row['areaName'] == 'Northeast Ohio' else 0, axis = 1)

# DFW
df['areaName_dfw'] = df.apply(lambda row: 1 if row['areaName'] == 'DFW' else 0, axis = 1)

# Austin
df['areaName_austin'] = df.apply(lambda row: 1 if row['areaName'] == 'Austin' else 0, axis = 1)

# San Antonio
df['areaName_san'] = df.apply(lambda row: 1 if row['areaName'] == 'San Antonio' else 0, axis = 1)

### <font color = blue>keep only prevstatus or status = confirmed

In [340]:
df = df[df.apply(lambda row: (row['prevStatus'] == 'confirmed') or (row['status'] == 'confirmed'), axis = 1)]

### <font color = green>output feature dataframe

In [341]:
df.columns

Index(['id', 'user_id', 'shift_id', 'status', 'createdAt',
       'hasNurseCheckEvent', 'hasBreakTime', 'prevStatus', 'distance', 'id.1',
       'facility_id', 'description', 'start_time', 'assigned_nurse_id', 'rate',
       'net_pay', 'unit', 'type', 'createdAt.1', 'qualifications', 'breakTime',
       'prevStatus.1', 'id.2', 'name', 'short_name', 'createdAt.2', 'rates',
       'S_Create', 'SA_Create', 'S_create2SA_Create', 'Start_Time',
       'SA_Create2Start_Time', 'S_Create2Start_Time', 'CW_Time',
       'CW_Time2Start_Time', 'SA_Create2CW_Time', 'S_Create2CW_Time',
       'allowedQualifications', 'areaId', 'areaName', 'segmentName',
       'U_create2now', 'U_approve2now', 'U_create2U_approve',
       'withdrawnInfo_key', 'withdrawnInfo_value', 'target', 'count_prev_SA',
       'count_prev_CW', 'prev_CW/SA_rate', 'prev_CW x SA_rate', 'type_RN',
       'type_LVN+LPN', 'segmentName_d', 'areaName_houston', 'areaName_no',
       'areaName_dfw', 'areaName_austin', 'areaName_san'],
    

In [343]:
feature_df = df[['id', 'user_id', 'shift_id', 'prev_CW/SA_rate', 'status', 'S_create2SA_Create',
                 'S_Create2Start_Time',
                 'SA_Create2Start_Time','U_create2now', 'U_approve2now','prev_CW x SA_rate', 'type_RN', 
                 'type_LVN+LPN', 'segmentName_d', 'areaName_houston', 'areaName_no', 'areaName_dfw', 
                 'areaName_austin', 'areaName_san', 'net_pay', 'distance','target', 'createdAt', 'Start_Time']]
feature_df.to_csv('model_data.csv')

In [ ]:
# def profit_calculation(model, x_value, y_value):
#     d_cutoff = 1/2

#     decision = list(model.predict_proba(x_value)[:,1] < d_cutoff)   
#     decision = list(model.predict(x_value))
#     y = list(y_value)
#     n_obs = len(y)

#     cum_profit = 0

#     for i in range(n_obs):
#         if decision[i] == True and y[i] == 1: 
#             profit = 0 
#         elif decision[i] == True and y[i] == 0: 
#             profit = 8 
#         elif decision[i] == False and y[i] == 1: 
#             profit = -2
#         else:
#             profit = 10 
#         cum_profit = cum_profit + profit 

#     average_net_profit = cum_profit / n_obs 
#     return average_net_profit